In [1]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import os
#from paths import *
import requests
import pandas as pd
import xarray as xr
from os import listdir
from datetime import datetime, timedelta, date
import contextlib

# Initialize the Earth Engine module
ee.Initialize()


#########################################################################
############################ USER INPUTS ################################
#########################################################################
# DOMAIN
# choose the modeling domain
domain = 'BEAU'
print(domain)
#path to CSO domains
domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
domains = domains_resp.json()

# PATHS
# path to temporary folder to store tif files from gee
TIFpath = domain +'_GEE_tmp/'

# TIME
# choose if want to set 'manual' or 'auto' date 
date_flag = 'manual'

BEAU


In [2]:
# Date setup function
def set_dates(st_dt,ed_dt,date_flag):
    if date_flag == 'auto':
        # ###automatically select date based on today's date 
        hoy = date.today()
        antes = timedelta(days = 2)
        #end date 3 days before today's date
        fecha = hoy - antes
        eddt = fecha.strftime("%Y-%m-%d") 
        #start date
        if fecha.month <10:
            styr = fecha.year - 1
        else:
            styr = fecha.year
        stdt = str(styr)+'-10-01'
    elif date_flag == 'manual':
        stdt = st_dt
        eddt = (datetime.strptime(ed_dt,'%Y-%m-%d') + timedelta(days = 1)).strftime('%Y-%m-%d')
    return stdt, eddt

In [3]:
# function to check for missing dates
def missing_slice_check(stdt, eddt, TIFpath):
    # create a 6-hourly timeseries with no missing values from the start to end date
    timesin = pd.date_range(start=stdt, end=eddt, freq='6H')[:-1]
    for time in timesin:
        nam = time.strftime('%Y%m%d%H')
        
    # compile list of all tif files downloaded from gee
    gee_times =[]

    for file in listdir(TIFpath):
        if file.endswith("tif"):
            datetmp = datetime.strptime(file[:-4], '%Y%m%d%H')
            gee_times.append(datetmp)
    gee_times = sorted(gee_times)

    # check for to see if all time slices downloaded from GEE
    if len(timesin) != len(gee_times):
        print('gee is missing timeslice(s):\n',timesin[~timesin.isin(gee_times)].values)
        
        # if 4 or more consecutive timeslices are missing - quit the function
        duration = []
        for i in range(len(gee_times)-1):
            time_delta = gee_times[i+1] - gee_times[i]
            duration.append(time_delta.total_seconds()/60/60)
        if max(duration) >= 24:    
            print('at least one full day of met data is missing - quitting function')

        # if there are less than 4 missing consecutive time slices 
        # repeat the met data from the last valid time slice 
        else:
            missingidx = np.where(~timesin.isin(gee_times))
            missing_idx = [int(val) for val in missingidx]
            missing_dt = timesin[missing_idx].tolist()
            for j in range(len(missing_dt)):
                if missing_idx[j] == 0:
                    print('choose earlier start date so missing time slices can be filled in')
                else:
                    pre_dt=TIFpath+timesin[missing_idx[j]-1].strftime('%Y%m%d%H')+'.tif'
                    mis_dt = TIFpath+timesin[missing_idx[j]].strftime('%Y%m%d%H')+'.tif' 
                    get_ipython().system('cp $pre_dt $mis_dt')
                    print('replaced', timesin[missing_idx[j]].strftime('%Y%m%d%H'),' with ', timesin[missing_idx[j]-1].strftime('%Y%m%d%H'))

In [4]:
# Format gee files for SnowModel function
def MET2SM(TIFpath, OUTpath, stdt, eddt):
    # create a 6-hourly timeseries with no missing values from the start to end date
    timesin = pd.date_range(start=stdt, end=eddt, freq='6H')[:-1]
    
    #load first tif to get dimensions
    ar = xr.open_rasterio(TIFpath+timesin[0].strftime('%Y%m%d%H')+'.tif')
    
    # empty arrays for each met variable
    T = np.empty((len(timesin),ar.shape[1],ar.shape[2]))
    Z = np.empty((len(timesin),ar.shape[1],ar.shape[2]))
    U = np.empty((len(timesin),ar.shape[1],ar.shape[2]))
    V = np.empty((len(timesin),ar.shape[1],ar.shape[2]))
    P = np.empty((len(timesin),ar.shape[1],ar.shape[2]))
    H = np.empty((len(timesin),ar.shape[1],ar.shape[2]))
    PR = np.empty((len(timesin),ar.shape[1],ar.shape[2]))

    # extract met data from tifs 
    for i in range(len(timesin)):

        #load tif file
        nam = TIFpath+timesin[i].strftime('%Y%m%d%H')+'.tif'
        ar = xr.open_rasterio(nam)
        T[i,:,:] = ar[0,:,:]
        Z[i,:,:] = ar[1,:,:]
        U[i,:,:] = ar[2,:,:]
        V[i,:,:] = ar[3,:,:]
        P[i,:,:] = ar[4,:,:]
        H[i,:,:] = ar[5,:,:]
        PR[i,:,:] = ar[6,:,:]
        

    #number of timesteps per dat 
    pointsperday = 4

    #compute number of grid points and time steps from size of 3d matrix
    t,y,x=PR.shape
    gridpts=x*y
    tsteps=t

    #create y m d h vectors
    year = timesin.year
    month = timesin.month
    day = timesin.day
    hour = timesin.hour

    #create ID numbers for the grid points
    ID=1000000+np.linspace(1,gridpts,gridpts)

    #create matrices of x and y values
    X, Y = np.meshgrid(ar.x.values, ar.y.values)
    X=X.flatten(order='F')
    Y=Y.flatten(order='F')

    #elevation is static (doesn't change with time)
    elev=Z[1,:,:].flatten(order='F')

    #find number of grid points with <0 elevation. Note: this is related to the
    #subroutine met_data_check in the preprocess_code.f. that subroutine seems
    #to suggest that negative elevations are ok (say, death valley). But, the
    #code itself checks for negative elevations and stops execution is any
    #negatives are found.
    I = np.where(elev>=0)
    validgridpts=np.shape(I)[1]

    #remove data at points with neg elevations
    ID=ID[I]
    X=X[I]
    Y=Y[I]
    elev=elev[I]

    #we are now ready to begin our main loop over the time steps.
    fid= open(OUTpath,"w+")

    for j in range(tsteps):
        #first we write the number of grid points
        fid.write('{0:6d}\n'.format(validgridpts))

        #prep data matrix for this time step. First, grab the jth time slice
        Prtmp=PR[j,:,:].flatten(order='F')
        Htmp=H[j,:,:].flatten(order='F')
        Ptmp=P[j,:,:].flatten(order='F')
        Ttmp=T[j,:,:].flatten(order='F')
        Utmp=U[j,:,:].flatten(order='F')
        Vtmp=V[j,:,:].flatten(order='F')

        #remove data at points with neg elevations
        Prtmp=Prtmp[I]
        Htmp=Htmp[I]
        Ptmp=Ptmp[I]
        Ttmp=Ttmp[I]
        Utmp=Utmp[I]
        Vtmp=Vtmp[I]


        #convert precip rate to precip DEPTH (mm) during time interval
        Prtmp=Prtmp*24*3600/pointsperday

        #convert specific hum. to RH from Clausius-Clapeyron. T is still in K
        RHtmp=0.263*Ptmp*Htmp*(np.exp(17.67*(Ttmp-273.16)/(Ttmp-29.65)))**(-1)

        #compute wind speed
        SPDtmp=np.sqrt(Utmp**2+Vtmp**2)

        #compute wind direction. 0-360, with 0 being true north! 90 east, etc.
        DIRtmp=np.degrees(np.arctan2(Utmp,Vtmp))
        K=np.where(DIRtmp>=180)
        J=np.where(DIRtmp<180)
        DIRtmp[K]=DIRtmp[K]-180
        DIRtmp[J]=DIRtmp[J]+180

        #put T in C
        Ttmp=Ttmp-273.16

        for z in range(len(Prtmp)): 

            fid.write('{:5.0f}\t'.format(int(year[j]))+'{:5.0f}\t'.format(int(month[j]))+
                      '{:3.0f}\t'.format(int(day[j]))+'{:6.3f}\t'.format(hour[j])+
                      '{:9.0f}\t'.format(int(ID[z]))+'{:12.1f}\t'.format(X[z])+
                      '{:12.1f}\t'.format(Y[z])+'{:8.1f}\t'.format(elev[z])+
                      '{:9.2f}\t'.format(Ttmp[z])+'{:9.2f}\t'.format(RHtmp[z])+
                      '{:9.2f}\t'.format(SPDtmp[z])+'{:9.2f}\t'.format(DIRtmp[z])+
                      '{:9.2f}\n'.format(Prtmp[z]))
    fid.close()

In [5]:
# Define some date info using the two lists
stdt = date(2018,10,1)
eddt = date(2020,9,30)
st_dt = stdt.strftime("%Y-%m-%d")
ed_dt = eddt.strftime("%Y-%m-%d")
print('getting met data for time period '+st_dt+' to '+ed_dt)
# path to where you want your output met .dat fime
OUTpath = '/nfs/attic/dfh/2020_NPRB/data/SMinputs/'+domain+'/mm_'+domain+'_wy'+str(stdt.year+1)+'-'+str(eddt.year)+'.dat'

# set time parameters
stdt, eddt = set_dates(st_dt,ed_dt,date_flag)


# download GEE data
#get_cfsv2(domain, TIFpath, stdt, eddt)


# fill in missing time slices or throw error if missing >4 slices
missing_slice_check(stdt, eddt, TIFpath)


# build SnowModel met file
MET2SM(TIFpath, OUTpath, stdt, eddt)


#     # delete directory with tif files 
#     get_ipython().system('rm -rf $TIFpath')

getting met data for time period 2018-10-01 to 2020-09-30
gee is missing timeslice(s):
 ['2020-04-03T18:00:00.000000000' '2020-08-10T06:00:00.000000000'
 '2020-08-10T12:00:00.000000000' '2020-08-10T18:00:00.000000000'
 '2020-08-11T00:00:00.000000000' '2020-08-11T06:00:00.000000000'
 '2020-08-11T12:00:00.000000000' '2020-08-11T18:00:00.000000000'
 '2020-08-12T00:00:00.000000000' '2020-08-12T06:00:00.000000000'
 '2020-08-12T12:00:00.000000000' '2020-08-12T18:00:00.000000000'
 '2020-08-13T00:00:00.000000000' '2020-08-13T06:00:00.000000000'
 '2020-08-13T12:00:00.000000000' '2020-08-13T18:00:00.000000000'
 '2020-08-14T00:00:00.000000000' '2020-08-14T06:00:00.000000000'
 '2020-08-14T12:00:00.000000000' '2020-08-14T18:00:00.000000000'
 '2020-08-15T00:00:00.000000000' '2020-08-15T06:00:00.000000000'
 '2020-08-15T12:00:00.000000000' '2020-08-15T18:00:00.000000000'
 '2020-08-16T00:00:00.000000000' '2020-08-16T06:00:00.000000000'
 '2020-08-16T12:00:00.000000000' '2020-08-16T18:00:00.000000000'
 '

KeyboardInterrupt: 